---
title: "Golomb ruler model"
date: 2023-06-24
type: docs
math: "true"
weight: 103
description: >
    Hell
---

# Golomb ruler problem
A Golomb ruler may be defined as a set of $m$ integers $0 = a_1 < a_2 < ... < a_m$ such that the $m(m-1)/2$ differences $a_j - a_i, 1 <= i < j <= m$ are distinct. Such a ruler is said to contain m marks and is of length $a_m$.

Let's first get the last version of [choco-solver](https://choco-solver.org/).

In [ ]:
%maven org.choco-solver:choco-solver:4.10.13

And manage imports.

In [ ]:
import org.chocosolver.solver.Model;
import org.chocosolver.solver.variables.IntVar;

import java.util.Arrays;

import static org.chocosolver.solver.search.strategy.Search.inputOrderLBSearch;

Now, first we declare $m$ to be the size of the ruler.

In [ ]:
int m = 10;

Then, we can start modelling the problem with choco.
The first step is to defined a `Model` instance.
It is required to declare and store the variables and the constraints.
For convenience, an instance can be declared with a name.

In [ ]:
Model model = new Model("GolombRuler");

Then, we need to declare the $a$ variables. 
In choco, any variable needs to be declared with a domain, sometimes setting the bounds is enough.
Here, $\forall i \in [1,m], \; a_i = [0, 9999]$ would do the job.

In [ ]:
IntVar[] ticks = model.intVarArray("a", m, 0, 9999);

The paramaters are:
- the prefix for setting the variables' name;
- the number of variables to create. Here, the method returns an `IntVar[]`;
- the lower bound and the upper bound of each variable.

Then, we can fix the value of $a_1$: 

In [ ]:
model.arithm(ticks[0], "=", 0).post();

And all the ticks are ordered:

In [ ]:
for (int i = 0; i < m - 1; i++) {
    model.arithm(ticks[i + 1], ">", ticks[i]).post();
}

The variables that encode the difference between ticks are defined as:

In [ ]:
IntVar[] diffs = model.intVarArray("d", (m * m - m) / 2, 0, 9999);

For convenience, we will also copy them to a matrix. That ease constraints declaration.

In [ ]:
IntVar[][] m_diffs = new IntVar[m][m];

Now, we can declare the required constraints: 

In [ ]:
for (int k = 0, i = 0; i < m - 1; i++) {
    for (int j = i + 1; j < m; j++, k++) {
        // d[k] is m[j]-m[i] and must be at least sum of first j-i integers
        model.scalar(new IntVar[]{ticks[j], ticks[i]}, new int[]{1, -1}, "=", diffs[k]).post();
        model.arithm(diffs[k], ">=", (j - i) * (j - i + 1) / 2).post();
        model.arithm(diffs[k], "-", ticks[m - 1], "<=", -((m - 1 - j + i) * (m - j + i)) / 2).post();
        model.arithm(diffs[k], "<=", ticks[m - 1], "-", ((m - 1 - j + i) * (m - j + i)) / 2).post();
        m_diffs[i][j] = diffs[k];
    }
}
model.allDifferent(diffs, "BC").post();

// break symetries
if (m > 2) {
    model.arithm(diffs[0], "<", diffs[diffs.length - 1]).post();
}

The Golomb ruler problem is defined as a Constraint Optimization Problem and thus an objective function has to be declared. The first parameter is the direction, the second is the variable to optimize:

In [ ]:
model.setObjective(Model.MINIMIZE, model.getVars()[m - 1]);

We have declared many variables, but finding values for each $a_i$ is enough, since the propagation will update the other ones' domain. 
These variables are known to be the decision ones and a search strategy can be set on them:  

In [ ]:
model.getSolver().setSearch(inputOrderLBSearch(ticks));

Everything is ready for the solving step to be executed.
We choose to display intermediate solutions by using a `while-loop`.

In [ ]:
while (model.getSolver().solve()) {
    System.out.printf("\nSolution #%d:\n", model.getSolver().getSolutionCount());
    Arrays.stream(ticks).forEach(t -> System.out.printf("%s = %d, ", t.getName(), t.getValue()));

}
model.getSolver().reset(); // to run the code twice